In [1]:
!pip install  nltk spacy scikit-learn seaborn wordcloud gensim vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 72.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.2
 

In [1]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 59.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
import pandas as pd

df = pd.read_csv('complaint_condensed.csv')

In [4]:
pd.DataFrame(df)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,03/21/2017,Credit reporting,NaN,Incorrect information on credit report,Information is not mine,NaN,Company has responded to the consumer and the ...,EXPERIAN DELAWARE GP,TX,77075,Older American,NaN,Phone,03/21/2017,Closed with non-monetary relief,Yes,No,2397100
1,04/19/2017,Debt collection,"Other (i.e. phone, health club, etc.)",Disclosure verification of debt,Not disclosed as an attempt to collect,NaN,NaN,"Security Credit Services, LLC",IL,60643,NaN,NaN,Web,04/20/2017,Closed with explanation,Yes,No,2441777
2,04/19/2017,Credit card,NaN,Other,NaN,NaN,Company has responded to the consumer and the ...,"CITIBANK, N.A.",IL,62025,NaN,NaN,Referral,04/20/2017,Closed with explanation,Yes,No,2441830
3,04/14/2017,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,Company believes it acted appropriately as aut...,"Shellpoint Partners, LLC",CA,90305,NaN,NaN,Referral,04/14/2017,Closed with explanation,Yes,No,2436165
4,04/19/2017,Credit card,NaN,Credit determination,NaN,NaN,Company has responded to the consumer and the ...,U.S. BANCORP,LA,70571,NaN,NaN,Postal mail,04/21/2017,Closed with explanation,Yes,No,2441726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,03/24/2014,Bank account or service,Checking account,"Account opening, closing, or management",NaN,NaN,NaN,"WELLS FARGO BANK, NATIONAL ASSOCIATION",NM,57936,Older American,NaN,Postal mail,03/31/2014,Closed with explanation,Yes,No,774267
24996,02/05/2013,Credit reporting,NaN,Credit reporting company's investigation,Problem with statement of dispute,NaN,NaN,EXPERIAN DELAWARE GP,IL,60914,NaN,NaN,Web,02/06/2013,Closed with explanation,Yes,No,292960
24997,08/14/2015,Credit reporting,NaN,Incorrect information on credit report,Information is not mine,NaN,NaN,"EQUIFAX, INC.",IL,60615,NaN,Consent not provided,Web,08/14/2015,Closed with explanation,Yes,No,1518864
24998,07/17/2016,Student loan,Non-federal student loan,Can't repay my loan,Can't temporarily postpone payments,NaN,NaN,"Navient Solutions, LLC.",WI,53219,NaN,Consent not provided,Web,07/17/2016,Closed with explanation,Yes,No,2016094


In [5]:
df = df[['Issue']].dropna().copy()
df

,Issue
0,Incorrect information on credit report
1,Disclosure verification of debt
2,Other
3,"Loan modification,collection,foreclosure"
4,Credit determination
...,...
24995,"Account opening, closing, or management"
24996,Credit reporting company's investigation
24997,Incorrect information on credit report
24998,Can't repay my loan


In [27]:
import spacy
import re
from nltk.corpus import stopwords

nlp = spacy.load('en_core_web_sm')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
  text = re.sub(r'[^a-zA-Z\s]','',text,re.I|re.A)
  text = text.lower()
  doc =nlp(text)
  lemmatized_tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and len(token.text.strip()) > 0]

  return " ".join(lemmatized_tokens)

df['cleaned_complaint'] = df['Issue'].apply(preprocess_text)
print(df[['Issue','cleaned_complaint']].head())

                                      Issue  \
0    Incorrect information on credit report   
1           Disclosure verification of debt   
2                                     Other   
3  Loan modification,collection,foreclosure   
4                      Credit determination   

                        cleaned_complaint  
0     incorrect information credit report  
1            disclosure verification debt  
2                                          
3  loan modificationcollectionforeclosure  
4                    credit determination  


In [28]:
df

,Issue,cleaned_complaint,sentiment_score,sentiment_label,rule_based_sentiment,ai_sentiment,final_sentiment,topic
0,Incorrect information on credit report,incorrect information credit report,0.3818,positive,Negative,NaN,Negative,0
1,Disclosure verification of debt,disclosure verification debt,-0.3612,negative,None,Neutral,Neutral,7
2,Other,,0.0000,neutral,None,Neutral,Neutral,23
3,"Loan modification,collection,foreclosure",loan modificationcollectionforeclosure,0.0000,neutral,None,Neutral,Neutral,1
4,Credit determination,credit determination,0.6486,positive,None,Neutral,Neutral,73
...,...,...,...,...,...,...,...,...
24995,"Account opening, closing, or management",account open closing management,0.0000,neutral,None,Neutral,Neutral,11
24996,Credit reporting company's investigation,credit reporting company investigation,0.3818,positive,None,Neutral,Neutral,20
24997,Incorrect information on credit report,incorrect information credit report,0.3818,positive,Negative,NaN,Negative,0
24998,Can't repay my loan,not repay loan,0.0000,neutral,None,Negative,Negative,21


In [29]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

def get_vader_sentiment(text):
  return analyser.polarity_scores(text)['compound']

df['sentiment_score'] = df['cleaned_complaint'].apply(get_vader_sentiment)

def classify_sentiment(score):
  if score >= 0.05:
    return 'positive'
  elif score <= -0.05:
    return 'negative'
  else :
    return 'neutral'

df['sentiment_label'] = df['sentiment_score'].apply(classify_sentiment)
print(df[['cleaned_complaint','sentiment_label']].head())

                        cleaned_complaint sentiment_label
0     incorrect information credit report        positive
1            disclosure verification debt        negative
2                                                 neutral
3  loan modificationcollectionforeclosure         neutral
4                    credit determination        positive


In [31]:
df.to_csv('intermediatecomplaint2.csv')

# Since the Sentiment Score are inconsistent and wrong at time due to limited vocabulary of VADER model so we have to make it more efficient :

## New Hybrid  Pipeline to increase the efficiency of the Sentiment Analysis :
### 1. Rule Based Check
### 2. FinBERT a finetuned model for finance for more context awareness
### 3. Categorization of Complaints using BERTopic
### 4. Final Analysis

In [32]:
import pandas as pd
import re

data = df

# 1. Rule Based Check

def rule_based_check(text):
  negative_keywords = [
    r'incorrect information',
    r'wrong amount',
    r'charged wrong',
    r'charged incorrectly',
    r'overcharged',
    r'over charge',
    r'overcharging',
    r'double charge',
    r'charged twice',
    r'duplicate charge',
    r'fraudulent charge',
    r'fraud charge',
    r'unauthorized charge',
    r'unauthorised charge',
    r'unexpected charge',
    r'unexpected fee',
    r'hidden fee',
    r'hidden charges',
    r'extra fee',
    r'excess fee',
    r'exorbitant fee',
    r'incorrect invoice',
    r'billing error',
    r'billing dispute',
    r'invoice error',
    r'wrong bill',
    r'bill incorrect',
    r'not my charge',
    r'not my transaction',
    r'mystery charge',
    r'mysterious charge',
    r'did not receive',
    r'not received',
    r'payment not received',
    r'payment not processed',
    r'payment failed',
    r'failed transaction',
    r'transaction failed',
    r'transaction declined',
    r'declined transaction',
    r'payment declined',
    r'payment reversed',
    r'reversed payment',
    r'payment not credited',
    r'not credited',
    r'credited to wrong account',
    r'applied to wrong account',
    r'missing funds',
    r'funds missing',
    r'lost funds',
    r'money missing',
    r'no record of payment',
    r'no trace of payment',
    r'uncleared funds',
    r'uncleared',
    r'payment pending too long',
    r'processing delay',
    r'delayed payment',
    r'processing taking too long',
    r'refund not received',
    r'refund denied',
    r'refund refused',
    r'refund delayed',
    r'no refund',
    r'partial refund',
    r'reimbursement not provided',
    r'chargeback denied',
    r'chargeback refused',
    r'refund pending',
    r'refund overdue',
    r'no reimbursement',
    r'unauthorized transaction',
    r'unauthorised transaction',
    r'unauthorized payment',
    r'account compromised',
    r'compromised account',
    r'identity theft',
    r'identity stolen',
    r'scam',
    r'scammed',
    r'scam attempt',
    r'phishing',
    r'phished',
    r'suspicious activity',
    r'fraudulent activity',
    r'defrauded',
    r'stolen card',
    r'stolen funds',
      r'card cloned',
    r'card compromised', r'defaulted charge', r'fake charge',
    r'locked out', r'account locked', r'cannot access account', r'unable to access',
    r'access denied', r'login failed', r'login error', r'login problem', r'password reset failed',
    r'password reset not working', r'2fa failed', r'two[- ]factor failed', r'blocked account',
    r'account blocked', r'account suspended', r'suspended account', r'closed my account without',
    r'on hold', r'on hold for an hour', r'on hold for \d+', r'long wait', r'long waiting',
    r'waiting for', r'waited', r'waited (too long|hours|minutes)', r'no response', r'no callback',
    r'no follow up', r'no follow-up', r'waiting time', r'very long delay', r'processing delay',
    r'customer service', r'poor service', r'bad service', r'unhelpful', r'not helpful',
    r'rude', r'rude staff', r'no resolution', r'no solution', r'ignored', r'not answered',
    r'hung up', r'cut me off', r'disconnected', r'refused to help', r'no support', r'no assistance',
    r'overdraft fee', r'insufficient funds fee', r'nsf fee', r'nsf', r'late fee',
    r'late payment fee', r'penalty fee', r'service charge', r'excessive charges',
    r'unjustified fee', r'unjustified charge', r'unauthorized fee', r'return fee', r'returned check fee',
    r'wrong account number', r'wrong routing number', r'wrong ifsc', r'incorrect details',
    r'wrong details', r'misapplied payment', r'applied incorrectly', r'entered wrong account',
    r'entered wrong amount', r'wrong beneficiary', r'incorrect beneficiary', r'bad reference',
    r'app crashed', r'app crash', r'website down', r'website error', r'gateway error',
    r'payment gateway error', r'system error', r'technical issue', r'technical problem',
    r'bug', r'glitch', r'system failure', r'session expired', r'failed to update', r'cannot complete transaction',
    r'cancelled but charged', r'charged after cancellation', r'keep charging after cancellation',
    r'cancel subscription but charged', r'autopay charged', r'auto-debit failed', r'automatically renewed',
    r'auto[- ]renew charged', r'standing instruction failed', r'stop payment not honoured',
    r'file a complaint', r'complaint filed', r'will report', r'will report to', r'reporting to regulator',
    r'ombudsman', r'legal action', r'sue', r'lawsuit', r'taking legal action', r'consumer court',
    r'frustrated', r'frustrating', r'disappointed', r'disappointing', r'angry', r'outraged',
    r'furious', r'worst experience', r'terrible', r'horrible', r'awful', r'not satisfied', r'never again',
    r'rip[- ]off', r'ripoff', r'scammy', r'scroll of abuse',
    r'not cleared', r'not settled', r'failed settlement', r'settlement failed', r'funds stuck',
    r'stuck in pending', r'unprocessed', r'not processed', r'cannot trace', r'cannot track payment',
    r'card declined', r'card blocked', r'card declined at', r'card denied', r'card not charged',
    r'bank error', r'bank mistake', r'bank refused', r'bank froze account',
    r'data breach', r'personal data exposed', r'leaked', r'privacy violation', r'unsecure',
    r'security issue', r'security breach', r'personal information leaked',
    r'credited incorrectly', r'credited to wrong account', r'credited late', r'credit delay',
    r'misposted', r'misapplied', r'post error', r'posted to wrong account',
    r'lost money', r'money gone', r'funds taken', r'funds withdrawn without', r'unjust charge',
    r'illegal charge', r'charge without consent', r'taken without permission', r'withdrawn without consent',
    r'refuse to refund', r'refused refund', r'refuse to reimburse', r'blocked funds',
    r'fraud', r'scam', r'chargeback', r'dispute', r'complaint', r'refund', r'overdraft', r'penalty',
    r'disconnected', r'ignored', r'unresponsive', r'unresolved',
    r'on hold for \d+\s*(minutes|minute|hours|hour|hrs|mins)',
    r'charged (twice|again|incorrectly|without permission)',
    r'(did not|didnt|didn\'t|not) (receive|get|see) (my|the) (payment|refund|money)',
    r'unauthorised', r'authorised', r'not recived', r'not recieve', r'not recieved',
    r'refundd', r'overchargd', r'duplicate charge', r'doube charge', r'chargd twice'
]


  text_lower = text.lower()

  for pattern in negative_keywords:
    if re.search(pattern , text_lower):
      return 'Negative'
  return None

df['rule_based_sentiment'] = df['Issue'].apply(rule_based_check)


In [33]:
pd.DataFrame(df)

,Issue,cleaned_complaint,sentiment_score,sentiment_label,rule_based_sentiment,ai_sentiment,final_sentiment,topic
0,Incorrect information on credit report,incorrect information credit report,0.3818,positive,Negative,NaN,Negative,0
1,Disclosure verification of debt,disclosure verification debt,-0.3612,negative,None,Neutral,Neutral,7
2,Other,,0.0000,neutral,None,Neutral,Neutral,23
3,"Loan modification,collection,foreclosure",loan modificationcollectionforeclosure,0.0000,neutral,None,Neutral,Neutral,1
4,Credit determination,credit determination,0.6486,positive,None,Neutral,Neutral,73
...,...,...,...,...,...,...,...,...
24995,"Account opening, closing, or management",account open closing management,0.0000,neutral,None,Neutral,Neutral,11
24996,Credit reporting company's investigation,credit reporting company investigation,0.3818,positive,None,Neutral,Neutral,20
24997,Incorrect information on credit report,incorrect information credit report,0.3818,positive,Negative,NaN,Negative,0
24998,Can't repay my loan,not repay loan,0.0000,neutral,None,Negative,Negative,21


# finBERT

In [12]:
!pip install transformers

In [34]:
from transformers import pipeline

sentiment_analyzer = pipeline('sentiment-analysis',model = "ProsusAI/finbert" , device=-1)
print('model loaded successfully')

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


model loaded successfully


In [35]:
def get_ai_sentiment(text):
  result = sentiment_analyzer(text)
  return result[0]['label'].capitalize()

mask = df['rule_based_sentiment'].isnull()
df.loc[mask , 'ai_sentiment'] = df.loc[mask , 'Issue'].apply(get_ai_sentiment)

print("\n--- After AI Sentiment Analysis ---")


--- After AI Sentiment Analysis ---


In [36]:
pd.DataFrame(df)

,Issue,cleaned_complaint,sentiment_score,sentiment_label,rule_based_sentiment,ai_sentiment,final_sentiment,topic
0,Incorrect information on credit report,incorrect information credit report,0.3818,positive,Negative,NaN,Negative,0
1,Disclosure verification of debt,disclosure verification debt,-0.3612,negative,None,Neutral,Neutral,7
2,Other,,0.0000,neutral,None,Neutral,Neutral,23
3,"Loan modification,collection,foreclosure",loan modificationcollectionforeclosure,0.0000,neutral,None,Neutral,Neutral,1
4,Credit determination,credit determination,0.6486,positive,None,Neutral,Neutral,73
...,...,...,...,...,...,...,...,...
24995,"Account opening, closing, or management",account open closing management,0.0000,neutral,None,Neutral,Neutral,11
24996,Credit reporting company's investigation,credit reporting company investigation,0.3818,positive,None,Neutral,Neutral,20
24997,Incorrect information on credit report,incorrect information credit report,0.3818,positive,Negative,NaN,Negative,0
24998,Can't repay my loan,not repay loan,0.0000,neutral,None,Negative,Negative,21


In [37]:
df['final_sentiment'] = df['rule_based_sentiment'].fillna(df['ai_sentiment'])
final_df = df[['Issue','final_sentiment']]

print('\n--- Final Hybrid Workflow Output ---')


--- Final Hybrid Workflow Output ---


In [38]:
pd.DataFrame(final_df)

,Issue,final_sentiment
0,Incorrect information on credit report,Negative
1,Disclosure verification of debt,Neutral
2,Other,Neutral
3,"Loan modification,collection,foreclosure",Neutral
4,Credit determination,Neutral
...,...,...
24995,"Account opening, closing, or management",Neutral
24996,Credit reporting company's investigation,Neutral
24997,Incorrect information on credit report,Negative
24998,Can't repay my loan,Negative


In [40]:
final_df.to_csv('finaldf2.csv')

# Topic Modelling and Clustering

In [41]:
!pip install bertopic

In [42]:
from bertopic import BERTopic

docs = df['cleaned_complaint'].tolist()

topic_model = BERTopic(verbose = True , embedding_model = "sentence-transformers/all-MiniLM-L6-v2")
topics , probs = topic_model.fit_transform(docs)

df['topic'] = topics

print(topic_model.get_topic_info())
print(topic_model.get_topic(0))

2025-09-30 13:26:26,187 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/782 [00:00<?, ?it/s]

2025-09-30 13:29:04,389 - BERTopic - Embedding - Completed ✓
2025-09-30 13:29:04,391 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-30 13:30:02,370 - BERTopic - Dimensionality - Completed ✓
2025-09-30 13:30:02,373 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-30 13:30:09,802 - BERTopic - Cluster - Completed ✓
2025-09-30 13:30:09,816 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-09-30 13:30:09,976 - BERTopic - Representation - Completed ✓


    Topic  Count                                               Name  \
0      -1     41               -1_stop_bank_reporting_investigation   
1       0   2141              0_incorrect_information_credit_report   
2       1   1712  1_modificationcollectionforeclosure_loan_prope...   
3       2   1399                 2_servicing_escrow_payment_account   
4       3   1377                        3_contd_owe_collect_attempt   
..    ...    ...                                                ...   
94     93     16           93_privacy_miss_incorrectmisse_confusing   
95     94     16                         94_acct_payment_not_credit   
96     95     15                      95_payoff_end_process_problem   
97     96     13                   96_delay_processing_application_   
98     97     10          97_reporting_investigation_company_credit   

                                       Representation  \
0   [stop, bank, reporting, investigation, company...   
1   [incorrect, information, cred

In [43]:
topic_model.get_topic(3)

[('contd', 0.09775966028953997),
 ('owe', 0.06422944574926552),
 ('collect', 0.06422944574926552),
 ('attempt', 0.06422944574926552),
 ('debt', 0.042438107859002504),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05)]

In [44]:
topic_sentiment_analysis = df.groupby('topic')['sentiment_label'].value_counts(normalize=True).unstack().fillna(0)
topic_sentiment_analysis['complaint_count'] = df['topic'].value_counts()
print(topic_sentiment_analysis)

sentiment_label  negative   neutral  positive  complaint_count
topic                                                         
-1               0.024390  0.000000   0.97561               41
 0               0.000000  0.000000   1.00000             2141
 1               0.000584  0.999416   0.00000             1712
 2               0.000000  1.000000   0.00000             1399
 3               1.000000  0.000000   0.00000             1377
...                   ...       ...       ...              ...
 93              0.250000  0.750000   0.00000               16
 94              0.312500  0.000000   0.68750               16
 95              1.000000  0.000000   0.00000               15
 96              1.000000  0.000000   0.00000               13
 97              0.000000  0.000000   1.00000               10

[99 rows x 4 columns]


In [45]:
topic_sentiment_analysis

sentiment_label,negative,neutral,positive,complaint_count
topic,,,,
-1,0.024390,0.000000,0.97561,41
0,0.000000,0.000000,1.00000,2141
1,0.000584,0.999416,0.00000,1712
2,0.000000,1.000000,0.00000,1399
3,1.000000,0.000000,0.00000,1377
...,...,...,...,...
93,0.250000,0.750000,0.00000,16
94,0.312500,0.000000,0.68750,16
95,1.000000,0.000000,0.00000,15


In [46]:
topic_sentiment_analysis.to_csv('topic_sentiment_analysis2.csv')

In [50]:
all_topics  = topic_model.get_topics()

In [51]:
topic_keywords_map = {}

for topic_num , topic_words in all_topics.items():
  top_words = [word for word , score in topic_words[:5] if word]
  topic_keywords_map[topic_num] = ", ".join(top_words)



In [55]:
topic_sentiment_analysis = topic_sentiment_analysis.reset_index()
topic_sentiment_analysis['Topic Keywords'] = topic_sentiment_analysis['topic'].map(topic_keywords_map)

topic_sentiment_analysis['Topic Keywords'].fillna('N/A',inplace = True)

In [57]:
topic_sentiment_analysis.to_csv('final_sentiment_analysis.csv')